# Space Charge Benchmark

Impact-Z only space charge benchmark from

C. Mayes, R. Ryne, D. Sagan
3D SPACE CHARGE IN BMAD
https://epaper.kek.jp/ipac2018/papers/thpak085.pdf

In [ ]:
import impact.z as IZ
from impact.z import ImpactZ, ImpactZInput, WriteFull, Drift

from scipy.constants import c

from pmd_beamphysics.units import mec2

import numpy as np

In [ ]:
energy0 = 10e6
gamma0 = energy0 / mec2
betagamma0 = np.sqrt(gamma0**2 - 1)
sigma_x0 = 1e-3
sigma_y0 = 1e-3
sigma_z0 = 0.1e-3

beta_x0 = 1000
beta_y0 = 100
n_particle = 100_000

In [ ]:
input = ImpactZInput(
    initial_particles=None,
    ncpu_y=1,
    ncpu_z=1,
    seed=-1,
    n_particle=n_particle,
    nx=32,
    ny=32,
    nz=32,
    distribution=IZ.DistributionType.gauss,
    twiss_beta_x=beta_x0,
    twiss_norm_emit_x=sigma_x0**2 * betagamma0 / beta_x0,
    twiss_beta_y=beta_y0,
    twiss_norm_emit_y=sigma_y0**2 * betagamma0 / beta_y0,
    average_current=0.13000000000000006,
    reference_kinetic_energy=9489001.04931,
    reference_particle_mass=510998.95069,
    reference_particle_charge=-1.0,
    reference_frequency=1300000000.0,
    lattice=[
        WriteFull(name="initial_particles", file_id=100),
        Drift(
            name="P1",
            metadata={"bmad_csr": False, "bmad_id": 1},
            length=1.0,
            steps=100,
            map_steps=100,
        ),
        WriteFull(name="final_particles", file_id=101),
    ],
)
input.set_twiss_z(0.1e-3 / c, 1)
input

In [ ]:
I = ImpactZ(input)
I.run()
P0 = I.output.particles["initial_particles"]
P1 = I.output.particles["final_particles"]

In [ ]:
P0 = I.output.particles["initial_particles"]
(
    P0["sigma_x"] / sigma_x0,
    P0["sigma_y"] / sigma_y0,
    P0["sigma_t"] * c / sigma_z0,
    P0["mean_energy"] / energy0,
)

In [ ]:
P1.plot("delta_t", "energy")

# Try again with particles

In [ ]:
I2 = ImpactZ(input, initial_particles=P0)
I2.run()
P3 = I2.output.particles["initial_particles"]
P4 = I2.output.particles["final_particles"]

In [ ]:
P4.plot("delta_t", "energy")